- A model that has been linked to the data is called "the conditioned model"
-  the order in which you place the lines of a @model macro matters

In [21]:
using Turing 
using StatsPlots
using StatsBase
using Optim

In [2]:
@model function gdemo(x, y)
    s² ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s²))
    x ~ Normal(m, sqrt(s²))
    y ~ Normal(m, sqrt(s²))
end

gdemo (generic function with 1 method)

In [14]:
#c1 = sample(gdemo(1.5, 2), SMC(), 1000)
#c2 = sample(gdemo(1.5, 2), PG(10), 1000)
#c3 = sample(gdemo(1.5, 2), HMC(0.1, 5), 1000)
# c4 = sample(gdemo(1.5, 2), Gibbs(PG(10, :m), HMC(0.1, 5, :s)), 1000) - doesn't work
# c5 = sample(gdemo(1.5, 2), HMCDA(0.15, 0.65), 1000)
c6 = sample(gdemo(1.5, 2), NUTS(0.65), n_chains=3,1000)

┌ Info: Found initial step size
│   ϵ = 1.6
└ @ Turing.Inference C:\Users\axs162731\.julia\packages\Turing\Gntg0\src\inference\hmc.jl:188


Chains MCMC chain (1000×14×1 Array{Float64, 3}):

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1
Samples per chain = 1000
parameters        = m, s²
internals         = acceptance_rate, hamiltonian_energy, hamiltonian_energy_error, is_accept, log_density, lp, max_hamiltonian_energy_error, n_steps, nom_step_size, numerical_error, step_size, tree_depth

Summary Statistics
  parameters      mean       std   naive_se      mcse        ess      rhat 
      Symbol   Float64   Float64    Float64   Float64    Float64   Float64 

           m    1.1653    0.8296     0.0262    0.0379   305.8115    0.9990
          s²    2.1529    2.0800     0.0658    0.1637   260.5822    0.9997

Quantiles
  parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
      Symbol   Float64   Float64   Float64   Float64   Float64 

           m   -0.4917    0.6658    1.1920    1.6791    2.7774
          s²    0.5882    1.1125    1.5720    2.3858    7.5673


In [29]:
@model function gdemo(x)
    s² ~ InverseGamma(2, 3)
    m ~ Normal(0, sqrt(s²))

    for i in eachindex(x)
        x[i] ~ Normal(m, sqrt(s²))
    end
end

# Create some data to pass to the model.
data = [1.5, 2.0]

# Instantiate the gdemo model with our data.
model = gdemo(data)
# Generate a MLE estimate.
mle_estimate = optimize(model, MLE())

ModeResult with maximized lp of -0.07
2-element Named Vector{Float64}
A   │ 
────┼───────
:s² │ 0.0625
:m  │   1.75

In [37]:
(mle_estimate)

ModeResult with maximized lp of -0.07
2-element Named Vector{Float64}
A   │ 
────┼───────
:s² │ 0.0625
:m  │   1.75

In [31]:
informationmatrix(mle_estimate)

2×2 Named Matrix{Float64}
A ╲ B │          :s²            :m
──────┼───────────────────────────
:s²   │   0.00390625  -7.62307e-14
:m    │ -7.62307e-14       0.03125

In [40]:
vi = Turing.VarInfo(model);